### Congrats!

Your previous work is proving useful, so your boss has asked to expand your analysis to the whole chain.

Let's get the new data: `all_stores.csv` under `data/`

# Imports

In [94]:
import os
import pandas as pd
import numpy as np
import hashlib # for grading purposes
%matplotlib inline

# Exercise 0 - Read the Data

In [95]:
# chain = ...

In [96]:
chain= pd.read_csv('all_stores.csv')
chain

,date,store_nbr,customers
0,23-05-2015,6,2053
1,17-02-2014,2,1766
2,26-03-2015,15,1336
3,14-09-2015,35,637
4,14-07-2013,44,4383
...,...,...,...
82673,25-07-2014,9,1761
82674,26-12-2014,27,2075
82675,03-07-2014,47,3784
82676,10-05-2014,13,1392


In [97]:
print('We now have %0.0f data points. Wooooow!' % len(chain))

We now have 82678 data points. Wooooow!


The thing is, we can't just set the index to be the day, as we now have multiple stores on the same day.

Looks like we have to go into multi-indexing...

## Exercise 1: Multi-indexing

#### 1.1) make the date into a datetime, and then set the index to be `[date, store_nbr]`

In [98]:
chain['date'] = pd.to_datetime(chain['date'],format='%d-%m-%Y')
chain = chain.set_index(['date', 'store_nbr'])
chain = chain.sort_index()
chain

customers
date       store_nbr           
2013-01-01 25               770
2013-01-02 1               2111
           2               2358
           3               3487
           4               1922
...                         ...
2017-07-31 50              2593
           51              1572
           52              2206
           53              1065
           54               860

[82678 rows x 1 columns]

In [99]:
assert hashlib.sha256(str(chain.index.get_level_values(0).dtype).encode()).hexdigest() == \
        '261738f2e43a1c47a16f043b46deb993943d61f4a2bbe5ef4b03c3fb1af362b5', "First level of index should be a datetime!"
assert hashlib.sha256(str(chain.index.get_level_values(0)).encode()).hexdigest() ==  \
        'ea3a4358e60ac9e478fd489b4ea9a3e2ebe0256e823b416125cf544160073b1f', "Is the index sorted? Never forget to sort your time series."
assert hashlib.sha256(str(chain.index.get_level_values(1)).encode()).hexdigest() ==  \
        'c602dc5ca179ea4f0f7d09e14dcd28a5cbdce1571acd6040357a513c0f70d53a', "Check if you selected the correct values for the second level."

AssertionError: Check if you selected the correct values for the second level.

#### 1.2) Customers on shop 10, April 2016

What's the maximum daily number of customers, on store 10, on April 2016?

In [108]:
# hint: the answer should be an integer
idx = pd.IndexSlice
max_store10 = chain.loc[idx[slice('2016-04-01','2016-04-30'), 10], 'customers'].max()
max_store10


1532

In [109]:
expected_hash = 'f76cb816b3f74ecf30d387c64869038ac163fe26f8aabd727c1071dd567fc3d5'
assert hashlib.sha256(str(max_store10).encode()).hexdigest() == expected_hash

print(f"Correct! The maximum daily number of customers on April 2016 was {max_store10}.")

Correct! The maximum daily number of customers on April 2016 was 1532.


#### 1.3) how many new stores opened in 2015, given that no existing stores closed in 2015?

In [149]:
# hint : Check number of stores at last day of each year.
nr_stores_2014 = chain.loc[idx[slice('2013-12-31','2014-12-31'), :], :].index.get_level_values(1).nunique()
nr_stores_2015 = chain.loc[idx[slice('2014-12-31','2015-12-31'), :], :].index.get_level_values(1).nunique()
nr_stores_opened_2015 = nr_stores_2015-nr_stores_2014



In [146]:
expected_hash = 'ef2d127de37b942baad06145e54b0c619a1f22327b2ebbcfbec78f5564afe39d'
assert hashlib.sha256(str(nr_stores_opened_2015).encode()).hexdigest() == expected_hash

#### 1.4) Record number of customers

Find the total number of customers in 2015 in each store. The result should be a pandas dataframe where the index corresponds to the `store_nbr` and the values to the corresponding sum of customers on record for 2015.

In [174]:
sum_per_store_2015 = chain.loc[idx[slice('2015-01-01','2015-12-31'), :], :].groupby('store_nbr')['customers'].sum()
sum_per_store_2015=sum_per_store_2015.to_frame()


In [175]:
assert sum_per_store_2015.shape[0] == 53, "There should be 53 stores in record_per_store's index."
expected_hash_1 = '6e941f790715b03b4634bdc0d844ab7743caaf05d633b71eb6e01f2efa95d777'
expected_hash_2 = 'a2fcf2b0fb1b2b76eac8ed32e88fb3de06d460db4e7c606efe3dc5dc09588a0a'
assert hashlib.sha256(str(sum_per_store_2015.iloc[5, :].values[0]).encode()).hexdigest() == expected_hash_1
assert hashlib.sha256(str(sum_per_store_2015.iloc[35, :].values[0]).encode()).hexdigest() == expected_hash_2

print(f"Good job!! Also, the store with the highest count of customers in 2015 was store {sum_per_store_2015.idxmax()[0]}, with a total of {sum_per_store_2015.max()[0]} customers. Now that's a lot of customers!...")

Good job!! Also, the store with the highest count of customers in 2015 was store 44, with a total of 1593523 customers. Now that's a lot of customers!...


/tmp/ipython-input-1968405151.py:7: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"Good job!! Also, the store with the highest count of customers in 2015 was store {sum_per_store_2015.idxmax()[0]}, with a total of {sum_per_store_2015.max()[0]} customers. Now that's a lot of customers!...")


---